In [1]:
import json

# Path to the original train.json file
train_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/train.json'

# Load the train.json file
with open(train_json_path, 'r') as file:
    train_data = json.load(file)

# Display the first few entries to verify the content
print(train_data[:3])  # Adjust the number of entries to display based on your need


[{'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What modality is used to take this image?', 'answer': 'MRI', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 0}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'Which part of the body does this image belong to?', 'answer': 'Abdomen', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Position', 'triple': ['vhead', '_', '_'], 'qid': 1}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What is the mr weighting in this image?', 'answer': 'T2', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 2}]


In [2]:
import os

def load_detection_json(img_folder):
    # Path to the folder containing detection.json
    detection_json_path = os.path.join('/Users/jeffreysherer/Dissertation/data/imgs-1', img_folder, 'detection.json')
    
    # Load the detection.json file
    if os.path.exists(detection_json_path):
        with open(detection_json_path, 'r') as file:
            detection_data = json.load(file)
        return detection_data
    else:
        return None  # Return None if file does not exist

# Test the function with an example folder
example_folder = 'xmlab1'
example_detection = load_detection_json(example_folder)
print(example_detection)  # Display the content of the detection.json for verification


[{'Liver': [54.0, 106.0, 30.0, 31.0]}]


In [3]:
# Function to add detection data to train_data entries
def integrate_detection_data(train_data):
    for item in train_data:
        # Extract the folder name from img_name
        folder_name = item['img_name'].split('/')[0]
        
        # Load detection data for the corresponding folder
        detection_data = load_detection_json(folder_name)
        
        # If detection data exists, add it to the train data item
        if detection_data:
            item['detection'] = detection_data
    
    return train_data

# Update train_data with detection information
updated_train_data = integrate_detection_data(train_data)

# Print a few updated entries to verify that detection data has been added correctly
print(updated_train_data[:3])  # Adjust the number of entries to display based on your need


[{'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What modality is used to take this image?', 'answer': 'MRI', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 0, 'detection': [{'Liver': [54.0, 106.0, 30.0, 31.0]}]}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'Which part of the body does this image belong to?', 'answer': 'Abdomen', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Position', 'triple': ['vhead', '_', '_'], 'qid': 1, 'detection': [{'Liver': [54.0, 106.0, 30.0, 31.0]}]}, {'img_id': 1, 'img_name': 'xmlab1/source.jpg', 'question': 'What is the mr weighting in this image?', 'answer': 'T2', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 2, 'detectio

In [4]:
import os
import json

# Base path where the original train.json is located
base_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0'

# Directory for the augmented files
augmented_dir = os.path.join(base_path, 'augmented')

# Create the augmented directory if it doesn't exist
if not os.path.exists(augmented_dir):
    os.makedirs(augmented_dir)

# Path to save the updated train.json file
updated_train_json_path = os.path.join(augmented_dir, 'updated_train.json')

# Write the updated train data to the new JSON file in the augmented directory
with open(updated_train_json_path, 'w') as file:
    json.dump(updated_train_data, file, indent=4)

print("Updated train.json has been saved to", updated_train_json_path)


Updated train.json has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_train.json


In [11]:
import json

# Load the augmented train.json file
augmented_train_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/updated_train.json'

try:
    with open(augmented_train_json_path, 'r') as file:
        data = json.load(file)

    # Reorder the dictionary so that 'detection' comes right before 'answer'
    new_data = []
    for entry in data:
        if 'detection' in entry:
            ordered_entry = {key: entry[key] for key in entry if key != 'answer'}
            ordered_entry['answer'] = entry['answer']
            new_data.append(ordered_entry)
        else:
            new_data.append(entry)

    # Path to save the corrected JSON file
    corrected_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/BBF_train.json'

    # Write the corrected data to a new JSON file
    with open(corrected_json_path, 'w') as file:
        json.dump(new_data, file, indent=4)

    print("Corrected train.json has been saved to", corrected_json_path)

except Exception as e:
    print("Error reading or processing the file:", str(e))


Corrected train.json has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/BBF_train.json


In [12]:
import json
import random

# Path to the corrected train.json file
corrected_train_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/BBF_train.json'

# Load the corrected train.json file
with open(corrected_train_json_path, 'r') as file:
    corrected_data = json.load(file)

# Randomly sample 5 entries from the dataset to inspect
sampled_entries = random.sample(corrected_data, 5)

# Print the sampled entries
for entry in sampled_entries:
    print(entry)


{'img_id': 205, 'img_name': 'xmlab205/source.jpg', 'question': 'Does the picture contain heart?', 'answer': 'Yes', 'q_lang': 'en', 'location': 'Chest_heart', 'modality': 'CT', 'answer_type': 'CLOSED', 'base_type': 'vqa', 'content_type': 'Organ', 'triple': ['vhead', '_', '_'], 'qid': 758}
{'img_id': 268, 'img_name': 'xmlab268/source.jpg', 'question': '图片中包含哪些疾病?', 'q_lang': 'zh', 'location': 'Abdomen', 'modality': 'CT', 'answer_type': 'OPEN', 'base_type': 'vqa', 'triple': ['vhead', '_', '_'], 'qid': 6379, 'content_type': 'Abnormality', 'detection': [{'Liver': [85.0, 133.0, 174.0, 128.0]}, {'Liver Cancer': [240.0, 149.0, 13.0, 16.0]}, {'Liver Cancer': [181.0, 174.0, 12.0, 11.0]}], 'answer': '肝癌'}
{'img_id': 271, 'img_name': 'xmlab271/source.jpg', 'question': 'Does this image look normal?', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'CT', 'answer_type': 'CLOSED', 'base_type': 'vqa', 'content_type': 'Abnormality', 'triple': ['vhead', '_', '_'], 'qid': 1470, 'detection': [{'Liver': 

Now doing the trst adn validate json files


In [13]:
import json
import os

# Paths to the original validate.json and test.json files
validate_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/validate.json'
test_json_path = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/test.json'

# Load the validate.json file
with open(validate_json_path, 'r') as file:
    validate_data = json.load(file)

# Load the test.json file
with open(test_json_path, 'r') as file:
    test_data = json.load(file)

# Display the first few entries to verify the content
print(validate_data[:3])  # Adjust the number of entries to display based on your need
print(test_data[:3])  # Adjust the number of entries to display based on your need


[{'img_id': 0, 'img_name': 'xmlab0/source.jpg', 'question': 'What modality is used to take this image?', 'answer': 'MRI', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 9835}, {'img_id': 0, 'img_name': 'xmlab0/source.jpg', 'question': 'Which part of the body does this image belong to?', 'answer': 'Abdomen', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Position', 'triple': ['vhead', '_', '_'], 'qid': 9836}, {'img_id': 0, 'img_name': 'xmlab0/source.jpg', 'question': 'What is the mr weighting in this image?', 'answer': 'T2', 'q_lang': 'en', 'location': 'Abdomen', 'modality': 'MRI', 'answer_type': 'OPEN', 'base_type': 'vqa', 'content_type': 'Modality', 'triple': ['vhead', '_', '_'], 'qid': 9837}]
[{'img_id': 102, 'img_name': 'xmlab102/source.jpg', 'question': 'What modality is used to take this image

In [14]:
# Directory containing the image folders with detection.json files
image_directory = '/Users/jeffreysherer/Dissertation/data/imgs-1'

# Function to update the data with bounding box coordinates
def update_data_with_bounding_boxes(data):
    for entry in data:
        img_folder = os.path.dirname(entry['img_name'])
        detection_json_path = os.path.join(image_directory, img_folder, 'detection.json')
        
        if os.path.exists(detection_json_path):
            with open(detection_json_path, 'r') as file:
                detection_data = json.load(file)
            entry['detection'] = detection_data

    return data

# Update the validate and test data
updated_validate_data = update_data_with_bounding_boxes(validate_data)
updated_test_data = update_data_with_bounding_boxes(test_data)


In [15]:
# Paths to save the updated validate and test JSON files
augmented_folder = '/Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented'
updated_validate_json_path = os.path.join(augmented_folder, 'BBF_validate.json')
updated_test_json_path = os.path.join(augmented_folder, 'BBF_test.json')

# Ensure the augmented folder exists
os.makedirs(augmented_folder, exist_ok=True)

# Write the updated validate data to a new JSON file
with open(updated_validate_json_path, 'w') as file:
    json.dump(updated_validate_data, file, indent=4)

# Write the updated test data to a new JSON file
with open(updated_test_json_path, 'w') as file:
    json.dump(updated_test_data, file, indent=4)

print("Updated validate.json has been saved to", updated_validate_json_path)
print("Updated test.json has been saved to", updated_test_json_path)


Updated validate.json has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/BBF_validate.json
Updated test.json has been saved to /Users/jeffreysherer/Desktop/Dissertation/Slake1.0/augmented/BBF_test.json
